# Advanced example

This guide shows how to analyze images of a real dataset jointly. The dataset consists of a MALDI and optical images of mouse urinary bladder. These are processed images from original images downloaded from PRIDE (dataset id [PXD001283](https://www.ebi.ac.uk/pride/archive/projects/PXD001283)). The original MALDI image was aligned on the same *m/z*-axis, following the procedure described in the [Data](data.rst) section. Then, the images were resized using the ``examples/resize_imzml.py`` script. Peak selection and alignment are already done, thus there is **no need for spectra processing** in the following example.

The optical image has been transformed by a 45° counter-clockwise rotation followed by a 100 px translation.

The resulting images are located in the ``data/Mouse_Urinary_Bladder_PXD001283/`` directory: ms_image_resized.imzML, optical_image.tiff. 

We aim at finding the spatial correlations between the ion images in MALDI and the optical image.

Let's import the necessary modules and define some utility functions:

In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import string
import SimpleITK as sitk

rootpath = "../../"
imagepath = "data/Mouse_Urinary_Bladder_PXD001283/"
sys.path.append(os.path.join(os.path.dirname(os.path.abspath("__file__")), rootpath))

def display_image(image):
    plt.axis("off")
    plt.imshow(image, cmap="gray")
    plt.show()

def display_images(*images):
    fig, ax = plt.subplots(1, len(images))
    for i in range(len(images)):
        ax[i].axis('off')
        ax[i].imshow(images[i], cmap="gray")
        ax[i].text(0.4, -0.2, "(" + string.ascii_lowercase[i] + ")", transform=ax[i].transAxes, size=18)
    plt.show()

## Input, Visualization

The data is available in the `data` directory. 
First, read the data and display the spectrum of the first pixel:

In [ ]:
import esmraldi.imzmlio as io

imzml = io.open_imzml(rootpath + imagepath + "ms_image_resized.imzML")
spectra = io.get_spectra(imzml)

print(spectra.shape)
mz_first, intensities_first = spectra[1, 0], spectra[1, 1]
indices = intensities_first > 1
plt.stem(mz_first[indices], intensities_first[indices])
plt.show()

In [ ]:
image = io.get_image(imzml, 741.7, 0.25)
display_image(image)

In [ ]:
from skimage import io as skio

optical_image = skio.imread(rootpath + imagepath + "optical_image.png", as_gray=True)
display_image(optical_image)

In [ ]:
import esmraldi.segmentation as seg

maldi_image = io.to_image_array(imzml)
maldi_image = np.transpose(maldi_image, (1, 0, 2))
maldi_image = io.normalize(maldi_image)

# Relevant images
spatially_coherent = seg.find_similar_images_area(maldi_image, 300, quantiles=[60, 70, 80, 90])

# Mean image
mean_image = np.average(spatially_coherent, axis=-1)

# Region growing
seeds = set(((1, 1), (0, 0)))
list_end, _ = seg.region_growing(spatially_coherent, seedList=seeds, lower_threshold=30)
x = [elem[0] for elem in list_end]
y = [elem[1] for elem in list_end]
mask = np.ones_like(mean_image)
mask[x, y] = 0
segmented_image = np.ma.array(mean_image, mask=mask)
segmented_image = segmented_image.filled(0)

display_image(segmented_image)

In [ ]:
import esmraldi.registration as reg

segmented_itk = sitk.Cast(sitk.GetImageFromArray(segmented_image), sitk.sitkFloat32)
optical_itk = sitk.Cast(sitk.GetImageFromArray(optical_image), sitk.sitkFloat32)
segmented_itk = seg.resize(segmented_itk, optical_itk.GetSize())
segmented_itk = sitk.Cast(segmented_itk, sitk.sitkFloat32)

number_bins = 8
sampling_percentage = 0.1
registration = reg.register(optical_itk, segmented_itk, number_bins, sampling_percentage, seed=0, learning_rate=0.8, relaxation_factor=0.8, min_step=0.0001)
registered_seg_itk = registration.Execute(segmented_itk)
registered_seg_image = sitk.GetArrayFromImage(registered_seg_itk)

display_images(optical_image, segmented_image, registered_seg_image)

In [ ]:
registered_itk = sitk.GetImageFromArray(maldi_image)
registered_itk = seg.resize(registered_itk, optical_itk.GetSize())
registered_itk = sitk.Cast(registered_itk, sitk.sitkFloat32)
size = registered_itk.GetSize()

size = [size[1], size[2], size[0]]
out_register = sitk.Image(size, registered_itk.GetPixelID() )
print(out_register.GetSize())
for i in range(registered_itk.GetSize()[0]):
    slice = registered_itk[i, :, :]
    slice.SetSpacing(optical_itk.GetSpacing())
    slice_registered = registration.Execute(slice)
    slice_registered = sitk.JoinSeries(slice_registered)
    out_register = sitk.Paste(out_register, slice_registered, slice_registered.GetSize(), destinationIndex=[0, 0, i])

registered_image = np.transpose(sitk.GetArrayFromImage(out_register), (1, 2, 0))

In [ ]:
plt.imshow(registered_image[..., 0])
plt.show()

In [ ]:
import esmraldi.fusion as fusion

optical_flatten = fusion.flatten(optical_image)
maldi_flatten = fusion.flatten(registered_image)

# Reduction by NMF
mzs = spectra[0, 0, :]
fit_red = fusion.nmf(maldi_flatten, n=7)
reduction = fit_red.transform(maldi_flatten)
point_optical = fit_red.transform(optical_flatten)
weights = [1 for i in range(reduction.shape[1])]

similar_images, similar_mzs, distances = fusion.select_images(registered_image, point_optical, reduction, weights, mzs, labels=None, top=None)

print("Closest m/z ratio ", similar_mzs[:3])
display_images(optical_image, similar_images[..., 0], similar_images[..., 1], similar_images[..., 2])